In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
%matplotlib inline
import numpy as np
import operator
from matplotlib import pyplot as plt
from repositories.CorpusRepository import *
from repositories.TokenRepository import *
from repositories.EstadisticasRepository import *

path = 'corpus'
corpusRepository = CorpusRepository(path)
tokenRepository = TokenRepository()
pathVacias = None

documentos = corpusRepository.getListDocuments()


In [2]:
response = tokenRepository.tokenizar(documentos,pathVacias = pathVacias)



100% ▏██████████████████████████████████████████████████▕



In [3]:
lista = sorted(response['terminos'].items(), key=operator.itemgetter(1) ,reverse=True)

frecuencias = []
for termino in lista:
    frecuencias.append(termino[1]['CF'])

ranking = xrange(1, len(frecuencias) + 1)


Defino una función que calcula la recta de ajuste de la distribución de frecuencias logarítmica y permite obtener la posición en el ranking de una frecuencia dada usando los coeficientes de dicha recta para despejar el valor buscado.

In [4]:
def obtener_ranking_de_frecuencia(ranking, frecuencias, frecuencia_buscada):
    x_log = np.log10(ranking)
    y_log = np.log10(frecuencias)
    recta_ajuste = np.polyfit(x_log, y_log, 1)
    pendiente = recta_ajuste[0]
    ordenada = recta_ajuste[1]
    frecuencia_buscada_log = np.log10(frecuencia_buscada)
    ranking_buscado_log = (frecuencia_buscada_log - ordenada) / float(pendiente)
    return int(pow(10, ranking_buscado_log))

Calculo la posición en el ranking para un término de frecuencia menor a 5.

In [5]:
ranking_frecuencia_menor_5 = obtener_ranking_de_frecuencia(ranking, frecuencias, 4)
print u"Posición: " + str(ranking_frecuencia_menor_5)

Posición: 3734


Defino una función que suma las frecuencias de todos los términos con un ranking igual o superior al establecido, para poder determinar qué porcentaje de las frecuencias seria omitido.

In [6]:
def calcular_porcentaje_de_frecuencia(frecuencias, ranking_buscado):
    frecuencia_total = sum(frecuencias)
    suma_frecuencia_buscada = 0
    for posicion, frecuencia in enumerate(frecuencias):
        if posicion + 1 >= ranking_buscado:
            suma_frecuencia_buscada += frecuencia
    return suma_frecuencia_buscada / float(frecuencia_total)

Calculo el porcentaje que estaría omitiendo según Zipf si eliminara los términos con frecuencia menor a 5.

In [7]:
porcentaje_frecuencia_menor_5 = calcular_porcentaje_de_frecuencia(frecuencias, ranking_frecuencia_menor_5) * 100
print "PORCENTAJE OMITIDO ZIPF: %.2f%%" % porcentaje_frecuencia_menor_5

PORCENTAJE OMITIDO ZIPF: 4.88%


Ahora defino una función que calcula realmente cual es frecuencia omitida al quitar todos los términos cuya frecuencia sea menor o igual a la establecida.

In [8]:
def calcular_porcentaje_de_frecuencia(frecuencias, frecuencia_buscada):
    frecuencia_total = sum(frecuencias)
    suma_frecuencia_buscada = 0
    for frecuencia in frecuencias:
        if frecuencia <= frecuencia_buscada:
            suma_frecuencia_buscada += frecuencia
    return suma_frecuencia_buscada / float(frecuencia_total)

Calculo el porcentaje que estoy omitiendo realmente al eliminar los términos con frecuencia menor a 5.

In [9]:
porcentaje_real_frecuencia_menor_5 = calcular_porcentaje_de_frecuencia(frecuencias, 4) * 100
print "PORCENTAJE OMITIDO REAL: %.2f%%" % porcentaje_real_frecuencia_menor_5

PORCENTAJE OMITIDO REAL: 5.06%


El porcentaje de términos realmente omitidos resulta ser muy similar al obtenido por la predicción de la ley de Zipf.